In [4]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [5]:
import os

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'cortex'

# Get keys from the environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if langchain_api_key:
    os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
else:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment.")

if groq_api_key:
    os.environ['GROQ_API_KEY'] = groq_api_key
else:
    raise ValueError("GROQ_API_KEY is not set in the environment.")

PART 12 - MULTI-REPRESENTATION INDEXING

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load documents from web URLs
loader1 = WebBaseLoader("https://medium.com/@pankaj_pandey/introduction-to-retrieval-augmented-generation-rag-9209bf8a076d")
docs = loader1.load()

loader2 = WebBaseLoader("https://medium.com/humansdotai/an-introduction-to-ai-agents-e8c4afd2ee8f")
docs.extend(loader2.load())

In [3]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | ChatGroq()
    | StrOutputParser()
)

summaries = chain.batch(docs, {"max_concurrency": 1})

In [4]:
summaries

['Retrieval-Augmented Generation (RAG) is an AI framework designed to improve the accuracy and reliability of Large Language Models (LLMs) by grounding them in external knowledge bases. RAG addresses the inconsistencies and errors in LLMs by providing access to up-to-date facts and verifiable sources, increasing user trust.\n\nRAG works in two phases: retrieval and content generation. In the retrieval phase, algorithms search for and retrieve relevant information from external knowledge bases based on the user\'s prompt or question. In the content generation phase, the retrieved information is appended to the user\'s prompt and fed to the LLM, which synthesizes a personalized answer based on both the augmented prompt and its internal knowledge base.\n\nRAG offers several advantages, including access to the latest, reliable facts, reduced opportunities for sensitive data leakage, and decreased need for continuous model retraining. It finds applications in personalized responses, verifia

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.documents import Document
import uuid

# Initialize HuggingFace embeddings with the updated class
model_name = "BAAI/bge-small-en"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Initialize the vectorstore for indexing child chunks with the updated class
vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=hf_embeddings
)

# Initialize the storage layer for parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# Create the retriever combining vectorstore and byte storage
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key
)

# Generate unique document IDs
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Create Document objects for summaries with linked metadata
summary_docs = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]})
    for i, summary in enumerate(summaries)
]

# Add summary documents to the vectorstore
retriever.vectorstore.add_documents(summary_docs)

# Link parent documents to their corresponding IDs in the byte store
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [9]:
doc_ids

['82983127-330e-4db2-a91d-ab74f8e28c7b',
 '3b093826-b9c1-4fbc-b488-21d663c8fcb3']

In [10]:
retriever.docstore.mget(doc_ids)

[Document(metadata={'source': 'https://medium.com/@pankaj_pandey/introduction-to-retrieval-augmented-generation-rag-9209bf8a076d', 'title': 'Introduction to Retrieval-Augmented Generation (RAG) | by Pankaj | Medium', 'description': 'The world is advancing rapidly, introducing new technologies and stacks in AI and other areas every day. Large Language Models (LLMs) are a significant innovation in this space. However, LLMs have…', 'language': 'en'}, page_content='Introduction to Retrieval-Augmented Generation (RAG) | by Pankaj | MediumOpen in appSign upSign inWriteSign upSign inMastodonIntroduction to Retrieval-Augmented Generation (RAG)Pankaj·Follow6 min read·Dec 16, 2023--ListenShareRAG systems aim to address the drawbacks of Large Language Models by incorporating factual information during response generation, mitigating issues such as knowledge cutoff and response hallucination.Retrieval Augmented Generation (RAG)The world is advancing rapidly, introducing new technologies and stacks

In [22]:
query = "What is agent"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

Document(id='9468469c-e9e1-49ab-8378-4ca9350b6dea', metadata={'doc_id': '3b093826-b9c1-4fbc-b488-21d663c8fcb3'}, page_content='AI Agents are intelligent digital systems that can perform tasks, make decisions, and interact with their environment like humans do. They are powered by machine learning, natural language processing, and other advanced technologies, allowing them to learn from data, adapt to new information, and execute complex functions autonomously. AI Agents exist in various forms, such as chatbots providing customer service and sophisticated robots used in healthcare and manufacturing. They are designed to understand, analyze, and respond to human input, constantly evolving to enhance their capabilities.\n\nThese agents navigate their environments and accomplish goals autonomously, free from human intervention. They address customer queries, make fast decisions based on real-time information, and simplify business processes and customer communications. AI Agents work by pe

In [23]:
retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
retrieved_docs[0].page_content[0:500]

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'An Introduction to AI Agents. Artificial Intelligence Agents are the… | by Humans.ai | humansdotai | MediumOpen in appSign upSign inWriteSign upSign inAn Introduction to AI AgentsHumans.ai·FollowPublished inhumansdotai·7 min read·Dec 27, 2023--2ListenShareArtificial Intelligence Agents are the digital newcomers revolutionizing our world. These agents, often called AI bots or virtual assistants, are intelligent systems programmed to perform tasks, make decisions, and interact with their environme'